# Finite-horizon Linear Quadratic Control of a Double Integrator


In this live script we use the linear quadratic regulator framework to control 
a double integrator system. This live scripts allows to change the parameters 
of this framework and therefore tune the behavior of the system (see lecture 
5).

Let us start by recalling the linear quadratic regulator framework. It is 
specified by a finite-horizon quadratic cost function

$$\sum_{k=0}^{h-1}x_k^{T}\,\,Qx_k+u_k^TRu_k+x_h^T Q_hx_h $$

and a linear model

$$ x_{k+1}=Ax_k+Bu_k$$

where $k\in\{0,\dots,h-1\}$. For the double integrator example, the state 
$x_k$ comprises position $y_k$ and velocity $v_k$, i.e., $x_k= [y_k \ \ v_k]^T$, 
and the control input $u_k$ is force. After discretizing the equation $\ddot{y}=u$ 
at a sampling period $\tau$ (zero order hold discretization) we obtain

$$A=e^{\left[ \matrix{ 0     & \tau \cr    0& 0 } \right]}={\left[ \matrix{ 
1    & \tau \cr    0& 1 } \right]} B = \int_0^\tau e^{As}ds\left[\matrix{0 
\cr 1} \right] = \left[\matrix{\frac{\tau^2}{2} \cr \tau} \right].$$

You can reproduce the behaviors for different cost functions and horizons 
by running the following code:

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
%matplotlib ipympl

In [ ]:
# define the parameters of the problem
Ac = np.array([[0, 1],[0, 0]])
Bc = np.array([[0],[1]])
tau = 0.2
A, B, C = signal.cont2discrete((Ac, Bc,  np.zeros((1,2)), np.array([0.2])), tau)[:3]
Q = np.array([[1, 0],[ 0, 1]])
S = np.array([[0],[0]])
R = 1
QT = np.array([[1, 0],[ 0, 1]])
h = 100
P = [ [] for i in range(h+1)]
K = [ [] for i in range(h+1)]
P[h] = QT

# iterate Riccati equations
for k in range(h-1,-1,-1):
    P[k] = A.T@P[k+1]@A + Q - (S+A.T@P[k+1]@B)*np.linalg.pinv(R+B.T@P[k+1]@B)@(S.T+B.T@P[k+1]@A)
    K[k] = -np.linalg.pinv(R+B.T@P[k+1]@B)@(S.T+B.T@P[k+1]@A)


# simulate the discrete-time system
x0 = np.array([[1],[0]])
t = tau*np.linspace(0,h,1)
n = 2
x = np.zeros((n,h+1))
x[:,[0]] = x0
u = np.zeros((1,h))
for k in range(h):
    u[:,[k]] = K[k]@x[:,[k]]
    x[:,[k+1]] = A@x[:,[k]]+B@u[:,[k]]

v = x[0,:]

# emulate a continuous-time simulation by running the system at a much
# higher frequency
N = 1000
ts = tau/N
nl = h*N
uc = np.kron(u,np.ones((1,N)))
Ad, Bd = signal.cont2discrete((Ac, Bc, np.zeros((1,A.shape[0])), np.zeros((1,Bc.shape[1]))), ts)[:2]
xc = np.zeros((n,nl+1))
xc[:,[0]] = x[:,[0]]
for l in range(nl):
    xc[:,[l+1]] = Ad@xc[:,[l]]+Bd*uc[0][l]


tc = ts*np.arange(0,nl+1)

In [ ]:
# plots
f = plt.figure(1)
ax = plt.gca()
ax.plot(tc,xc[0,:])
ax.grid(True)
ax.set_xlabel('t');

In [ ]:
f = plt.figure(2)
ax = plt.gca()
ax.plot(tc,xc[1])
ax.grid(True)
ax.set_xlabel('t');

In [ ]:
f = plt.figure(3)
ax = plt.gca()
ax.plot(tc[:-1], uc[0])
ax.grid(True)
ax.set_xlabel('t');

In [ ]:
f, ax = plt.subplots(1,3)
ax[0].plot(tc, xc[0])
ax[0].set_xlabel('t')
ax[0].set_ylabel('y(t)')
ax[0].grid(True)

ax[1].plot(tc, xc[1])
ax[1].set_xlabel('t')
ax[1].set_ylabel('v(t)')
ax[1].grid(True)

ax[2].plot(tc[:-1], uc[0])
ax[2].grid(True)
ax[2].set_xlabel('t')
ax[2].set_ylabel('u(t)');

cost:

In [ ]:
x0.T@P[0]@x0